> **WARNING** It is more than recommended to work on a copy of that file (_i.e._ not IN the cloned git repository) otherwise next time you update the repo with `git pull` you'll face a `merging` issue and these are tricky to manage

# My first landscape

In this notebook, we will use the basic `trackscape` module, aka without the python helper, to run from scratch a simple fluvial landscape.

## Importing the right codes

First let's import the different modules we will use:

- matplotlib for the visualisation
- numpy for generating input data
- popscape_dagger for the model
- Note the `%matplotlib wdiget` is an optional widget to get interactive figures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import dagger as dag


%matplotlib widget



## Setting up parameters

For the sake of clarity, let's set up the param before running the model. This is a basic example, so we will simply set homogeneous model parameters. The values of the parameters will be discussed in a later tutorial:

- `ny,nx` are the number of pixels in the y and x directions
- `dy, dx` are the spacing between two pixels in the y and x directions. The length in therefore `(ny + 1) * dy` and `(nx + 1) * dx` if we assume the coordinates at the center of the pixels
- `rshp` represents the **shape** of the 2D numpy arrays of model dimensions. Any `DAGGER` related codes process the landscapes as a big 1D array, visualising the data as an image requires reshaping the 1D array in a 2D matrix
- `dt` is the model time step (time interval between each calculations)
- `Urate` is the uplift rate of the model
- `Kr` is the bedrock erodability
- `Ks` is the sediment erodability
- `dep` is the deposition coefficient

In [ ]:
ny,nx = 256,256
dy,dx = 200,200
Urate = 5e-4
dt = 500
Kr = 1e-5
Ks = 2e-5
dep = 4
rshp = (ny,nx)

## Setting up model object

`python` and `c++` are object oriented lanuages. An object is a set of data and function associated together to operate smoothly, and that's where we control the model behavior. 

Note here that we initialise the model with white noise - further options will come soon. We will also demonstrate other boundary conditions, `"4edges"` means that all the boundaries outlets the model.

For this first example, we also set the hillslope processes off. By default, the model is set to run in Single Flow Direction mode.

In [ ]:
# Initialising an empty model in the variable ts
ts = dag.trackscape()
# Initialising the topography and its dimensions
ts.init_random(dag.NOISE.WHITE, nx, ny,dx,dy,"4edges")

# FUnctions to set parameters as global homogeneous values (if not initialised, there is a default value)
ts.set_single_Kr(Kr)
ts.set_single_Ks(Ks)
ts.set_single_depcoeff(dep)

# Deactivate hilllsopes processes
ts.hillslopes_off()

## Running the model

Finally we can run the model. The model has a `run` function that automatically runs one timestep with the options inputted. We set up an interractive plot, this is not the most instinctive way to visualise, but it is satisfying. Another notebook will detail the different outputs.


This first cell generates the figure:

In [ ]:
fig,ax = plt.subplots()

topo = ts.get_topo().reshape(rshp)
im = ax.imshow(topo, cmap = "gist_earth")
plt.colorbar(im, label = "elevation (m)")

**Let's now run the model `ndt` times, and update the figure every `nupdate` timesteps** (updating the figure too often can be costly).

> **Note** the figure is updated in place, just above. If you deactivated the widget, it won't work

In [ ]:
ndt = 10000
nupdate = 100

# Main loop
for i in range(ndt):
    # Calling the run function for single flow
    ts.run_SFD(dt)
    # Calling hte block uplift funtion)
    ts.block_uplift(Urate,dt)
    
    # If nupdate^th timestep: I update the fig
    if(i%nupdate == 0):
        ## printing the timestep
        print("                   ",end = "\r")
        print("Timestep",i,end = "\r")
        
        #GEtting the topography
        topo = ts.get_topo().reshape(rshp)
        # Setting the new data
        im.set_data(topo)
        # Setting the new colorbar limits
        im.set_clim(topo.min(), topo.max())
        # Apply the update
        fig.canvas.draw()
        
        
        
        

In [ ]:
ts.fluvial_off()